# Setup

In [1]:
try:
    %tensorflow_version 2.x
except:
    pass

In [1]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf


print('\u2022 Using TensorFlow Version:', tf.__version__)
print('\u2022 GPU Device Found.' if tf.test.is_gpu_available() else '\u2022 GPU Device Not Found. Running on CPU')

• Using TensorFlow Version: 2.8.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
• GPU Device Found.


# Mount Drive to Access The Data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd drive/My\ Drive/Dataset/
!ls

/content/drive/My Drive/Dataset
train  validation


The class names are not included with the dataset, so we will specify them here.

In [6]:
import os

akel_dir = os.path.join('train/Akar Kelapa')
bakmi_dir = os.path.join('train/Bakmi')
bakso_dir = os.path.join('train/Bakso')
bangkit_dir = os.path.join('train/Kue Bangkit')
tambang_dir = os.path.join('train/Kue Tambang')
nasgor_dir = os.path.join('train/Nasi Goreng')
nastar_dir = os.path.join('train/Nastar')
onde_dir = os.path.join('train/Onde Onde')
rendang_dir = os.path.join('train/Rendang')
sate_dir = os.path.join('train/Sate')
semur_dir = os.path.join('train/Semur Tahu')
tempe_dir = os.path.join('train/Tempe Orek')

print('total training akar kelapa images:', len(os.listdir(akel_dir)))
print('total training bakmi images:', len(os.listdir(bakmi_dir)))
print('total training bakso images:', len(os.listdir(bakso_dir)))
print('total training kue bangkit images:', len(os.listdir(bangkit_dir)))
print('total training kue tambang images:', len(os.listdir(tambang_dir)))
print('total training nasi goreng images:', len(os.listdir(nasgor_dir)))
print('total training nastar images:', len(os.listdir(nastar_dir)))
print('total training onde onde images:', len(os.listdir(onde_dir)))
print('total training rendang images:', len(os.listdir(rendang_dir)))
print('total training sate images:', len(os.listdir(sate_dir)))
print('total training semur tahu images:', len(os.listdir(semur_dir)))
print('total training tempe images:', len(os.listdir(tempe_dir)))

total training akar kelapa images: 30
total training bakmi images: 30
total training bakso images: 30
total training kue bangkit images: 30
total training kue tambang images: 30
total training nasi goreng images: 30
total training nastar images: 30
total training onde onde images: 30
total training rendang images: 30
total training sate images: 30
total training semur tahu images: 30
total training tempe images: 30


In [ ]:
# The images in the dataset are 28 by 28 pixels.
# IMG_SIZE = 28

# Preprocessing data

## Preprocess

In [ ]:
# # EXERCISE: Write a function to normalize the images.

# def format_example(image, label):
#     # Cast image to float32
#     image = # YOUR CODE HERE
        
#     # Normalize the image in the range [0, 1]
#     image = # YOUR CODE HERE
    
#     return image, label

In [ ]:
# Specify the batch size
# BATCH_SIZE = 256

# Building the Model

```
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
flatten (Flatten)            (None, 3872)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                247872    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
=================================================================
Total params: 253,322
Trainable params: 253,322
Non-trainable params: 0
```

In [9]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "train/"
training_datagen = ImageDataGenerator(
      rescale = 1.0/255.,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "validation/"
validation_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical'
)

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(12, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=25 , validation_data = validation_generator, verbose = 1)

Found 360 images belonging to 12 classes.
Found 122 images belonging to 12 classes.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 34, 34, 128)       73856     
                                    

In [12]:
#Mengurangi dan memodifikasi layer (So far, ini yang pas walau acc-nya masih kecil)
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(12, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=25 , validation_data = validation_generator, verbose = 1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 74, 74, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 17, 17, 128)     

In [10]:
#Mengurangi epochs
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = model.fit(train_generator, epochs=10 , validation_data = validation_generator, verbose = 1)

Epoch 1/10
12/12 [==============================] - 6s 475ms/step - loss: 2.2142 - accuracy: 0.3833 - val_loss: 1.7140 - val_accuracy: 0.3934
Epoch 2/10
12/12 [==============================] - 5s 449ms/step - loss: 1.5534 - accuracy: 0.4556 - val_loss: 1.3647 - val_accuracy: 0.4836
Epoch 3/10
12/12 [==============================] - 5s 444ms/step - loss: 1.7200 - accuracy: 0.4444 - val_loss: 1.4148 - val_accuracy: 0.4918
Epoch 4/10
12/12 [==============================] - 5s 446ms/step - loss: 1.4343 - accuracy: 0.4861 - val_loss: 1.3930 - val_accuracy: 0.4508
Epoch 5/10
12/12 [==============================] - 5s 451ms/step - loss: 1.4675 - accuracy: 0.4972 - val_loss: 2.2483 - val_accuracy: 0.4180
Epoch 6/10
12/12 [==============================] - 5s 446ms/step - loss: 1.6602 - accuracy: 0.4500 - val_loss: 1.4488 - val_accuracy: 0.4344
Epoch 7/10
12/12 [==============================] - 5s 471ms/step - loss: 1.4388 - accuracy: 0.5000 - val_loss: 1.4672 - val_accuracy: 0.4426
Epoch 